**When will the United States reach herd immunity (>230M) for COVID-19?**

Goal: Preict the date of when the first reliable media report is published that states that >230M of the US population (~70%) have either received a SARS-CoV-2 vaccine or has been previously been infected by the virus.

Factors:
* COVID case growth rates
* Vaccines timeline (getting EUA, starting distribution, achieving sufficient scale)
* Delay between achievement and the media article

In [124]:
import math
import random

import pandas as pd
import numpy as np

from scipy import stats
from datetime import datetime, timedelta


# Model variables
GROWTH_DERIVATIVE_FACTOR_LOW = 5
GROWTH_DERIVATIVE_FACTOR_HIGH = 30

MONTHLY_GROWTH_MODIFIER_LOW = 0.5
MONTHLY_GROWTH_MODIFIER_HIGH = 4.0

DAYS_UNTIL_PEAK_GROWTH_LOW = -30
DAYS_UNTIL_PEAK_GROWTH_HIGH = 60

VACCINE_MODIFIER_LOW = 0.5
VACCINE_MODIFIER_HIGH = 1.5

RANDOM_CASE_FLUCTUATION_LOW = -2000
RANDOM_CASE_FLUCTUATION_HIGH = 4000

DAILY_FOURTH_WAVE_CHANCE = 0.6/365
FOURTH_WAVE_LENGTH_DAYS_LOW = 30
FOURTH_WAVE_LENGTH_DAYS_HIGH = 120

million = 1000000

# These are set to roughly curve fit the estimates from
# https://www.metaculus.com/questions/4066/10m-are-administered-an-efficacious-vaccine/
# https://www.metaculus.com/questions/5727/100m-people-vaccinated-against-covid-19/
# https://www.metaculus.com/questions/5767/1bn-people-vaccinated-against-covid-19/
MONTHLY_VACCINATION_DAILY_AMOUNTS = [np.round(5 * million * (1/30) * (1/12), 0),
                                     np.round(20 * million * (1/30) * (1/12), 0),
                                     np.round(25 * million * (1/30) * (1/12), 0),
                                     np.round(35 * million * (1/30) * (1/12), 0),
                                     np.round(40 * million * (1/30) * (1/12), 0),
                                     np.round(60 * million * (1/30) * (1/12), 0),
                                     np.round(75 * million * (1/30) * (1/12), 0),
                                     np.round(100 * million * (1/30) * (1/12), 0),
                                     np.round(150 * million * (1/30) * (1/12), 0),
                                     np.round(200 * million * (1/30) * (1/12), 0),
                                     np.round(275 * million * (1/30) * (1/12), 0),
                                     np.round(300 * million * (1/30) * (1/12), 0),
                                     np.round(330 * million * (1/30) * (1/12), 0),
                                     np.round(370 * million * (1/30) * (1/12), 0),
                                     np.round(420 * million * (1/30) * (1/12), 0),
                                     np.round(480 * million * (1/30) * (1/12), 0),
                                     np.round(560 * million * (1/30) * (1/12), 0)]

# Simulation parameters
N_SCENARIOS = 1000
SIM_START_DATE = '2020-11-28'
SIM_END_DATE = '2022-12-21'
CURRENT_COVID_CASES = 13.6 * million
CURRENT_COVID_GROWTH = 158000
CURRENT_COVID_DERIVATIVE = 2000
Q4_2020_START_COVID = 6.03 * million
TARGET_POP = 230 * million
US_POP = 328.2 * million

In [125]:
def print_pct(pct, digits=0):
    pct = pct * 100
    pct = np.round(pct, digits)
    if pct >= 100:
        if digits == 0:
            val = '>99.0%'
        else:
            val = '>99.'
            for d in range(digits - 1):
                val += '9'
            val += '9%'
    elif pct <= 0:
        if digits == 0:
            val = '<1.0%'
        else:
            val = '<0.'
            for d in range(digits - 1):
                val += '0'
            val += '1%'
    else:
        val = '{}%'.format(pct)
    return val


def get_metaculus_range(rangex, minx=None, maxx=None):
    lower25, mean, upper75 = np.percentile(rangex, [25, 50, 75])
    results = {}
    results['lower25'] = lower25
    results['mean'] = mean
    results['upper75'] = upper75
    if minx:
        results['minx'] = print_pct(sum([x < minx for x in rangex]) / len(rangex))
    if maxx:
        results['maxx'] = print_pct(sum([x > maxx for x in rangex]) / len(rangex))
    return results
    

def normal_sample(low, high, interval):
    if (low > high) or (high < low):
        raise ValueError
    if low == high:
        return low
    else:
        mu = (high + low) / 2
        cdf_value = 0.5 + 0.5 * interval
        normed_sigma = stats.norm.ppf(cdf_value)
        sigma = (high - mu) / normed_sigma
        return np.random.normal(mu, sigma)

    
def lognormal_sample(low, high, interval):
    if (low > high) or (high < low):
        raise ValueError
    if low == high:
        return low
    else:
        log_low = np.log(low)
        log_high = np.log(high)
        mu = (log_high + log_low) / 2
        cdf_value = 0.5 + 0.5 * interval
        normed_sigma = stats.norm.ppf(cdf_value)
        sigma = (log_high - mu) / normed_sigma
        return np.random.lognormal(mu, sigma)

In [126]:
days_until_sim_end_date = (datetime.strptime(SIM_END_DATE, '%Y-%m-%d') - datetime.strptime(SIM_START_DATE, '%Y-%m-%d')).days + 1

dec_7_2020_covid_collector = []
y2020_covid_collector = []
q1_2021_covid_collector = []
q2_2021_covid_collector = []
q3_2021_covid_collector = []
q4_2021_covid_collector = []
q1_2021_vaccinated_collector = []
q2_2021_vaccinated_collector = []
q3_2021_vaccinated_collector = []
q4_2021_vaccinated_collector = []
q1_2021_target_collector = []
q2_2021_target_collector = []
q3_2021_target_collector = []
q4_2021_target_collector = []
target_date_collector = []


for n in range(N_SCENARIOS):
    done = False
    fourth_wave = False
    today = datetime.today()
    
    covid = CURRENT_COVID_CASES
    covid_growth = CURRENT_COVID_GROWTH
    
    growth_derivative_factor = np.round(normal_sample(GROWTH_DERIVATIVE_FACTOR_LOW, GROWTH_DERIVATIVE_FACTOR_HIGH, 0.8) + 1, 1)
    if growth_derivative_factor < 0.3:
        growth_derivative_factor = 0.3
    
    monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
    if monthly_growth_modifier < 0.5:
        monthly_growth_modifier = 0.5
     
    peak_growth_days = np.round(normal_sample(DAYS_UNTIL_PEAK_GROWTH_LOW, DAYS_UNTIL_PEAK_GROWTH_HIGH, 0.8), 0)
    if peak_growth_days < 0:
        peak_growth_days = 0
    peak_growth_date = (today + timedelta(days=peak_growth_days)).date()
    
    fourth_wave_days_length = np.round(lognormal_sample(FOURTH_WAVE_LENGTH_DAYS_LOW, FOURTH_WAVE_LENGTH_DAYS_HIGH, 0.8), 0)
    if fourth_wave_days_length < 1:
        fourth_wave_days_length = 1
        
    vaccine_modifier = np.round(normal_sample(VACCINE_MODIFIER_LOW, VACCINE_MODIFIER_HIGH, 0.8), 1)
    if vaccine_modifier < 0.3:
        vaccine_modifier = 0.3
    
    vaccine_rate = 0
    total_vaccinated = 0
    
    verbose = (n <= 4) or (n % 10 == 0)
    really_verbose = (n <= 4)
    
    if verbose:
        print('-')
        print('-')
        print('## SCENARIO {}/{} ##'.format(n + 1, N_SCENARIOS))
        print('Growth derivative factor: {}'.format(growth_derivative_factor))
        print('Peak growth on: {}'.format(peak_growth_date))

    for day in range(days_until_sim_end_date):
        date = str((today + timedelta(days = day)).date())
        if really_verbose:
            print('Day {}: {}'.format(day, date))
            
        # Grow vaccines
        days_since_vaccine_start = (today + timedelta(days = day) - datetime(2021, 1, 1)).days - vaccine_delay_days
        if days_since_vaccine_start >= 0:
            i = math.floor(days_since_vaccine_start / 30)
            if i >= len(MONTHLY_VACCINATION_DAILY_AMOUNTS):
                i = len(MONTHLY_VACCINATION_DAILY_AMOUNTS) - 1
            daily_vaccines = MONTHLY_VACCINATION_DAILY_AMOUNTS[i]
            old_vaccine_rate = vaccine_rate
            vaccine_rate = np.round(np.random.normal(daily_vaccines  * vaccine_modifier,
                                                     daily_vaccines  * vaccine_modifier), 0)
            if vaccine_rate < old_vaccine_rate:
                vaccine_rate = old_vaccine_rate
            total_vaccinated = np.round(total_vaccinated + vaccine_rate, 0)
            
        if really_verbose:
            print('...Total vaccinated now {} (+{})'.format(total_vaccinated, vaccine_rate))
            
        if really_verbose:
            print('...Total vaccinated now {} (+{})'.format(total_vaccinated, vaccine_rate))
            
            
        # Grow COVID
        if day > peak_growth_days and (not fourth_wave or day > fourth_wave_end):
            covid_growth -= CURRENT_COVID_DERIVATIVE / growth_derivative_factor * (1 / monthly_growth_modifier)
        else:
            covid_growth += (CURRENT_COVID_DERIVATIVE / growth_derivative_factor) * monthly_growth_modifier
        covid_growth = np.round(covid_growth, 0)
        old_covid_growth = covid_growth
        if old_covid_growth < 0:
            old_covid_growth = 0
            
        covid_growth -= np.round(vaccine_rate * 0.02, 0)
        if covid_growth < 0:
            covid_growth = 0
            
        random_case_fluctuation = np.round(normal_sample(RANDOM_CASE_FLUCTUATION_LOW, RANDOM_CASE_FLUCTUATION_HIGH, 0.8), 0)
        covid += covid_growth + random_case_fluctuation
        current_pop = covid + total_vaccinated
        current_target = current_pop / US_POP * 100
        if current_pop >= TARGET_POP and not done:
            target_date_collector.append(day)
            done = True
            
        if really_verbose:
            print('...Total COVID cases now {} (+{} {} -{} *{})'.format(covid,
                                                                        old_covid_growth,
                                                                        "%+d" % random_case_fluctuation,
                                                                        np.round(vaccine_rate * 0.02, 0),
                                                                        monthly_growth_modifier))
            print('...% of target: {}%/{}%'.format(np.round(current_target, 1),
                                                   np.round(TARGET_POP / US_POP * 100, 1)))
                
        if not fourth_wave and np.random.random() <= DAILY_FOURTH_WAVE_CHANCE:
            if really_verbose:
                print('...COVID FOURTH WAVE!!!')
            fourth_wave = True
            fourth_wave_end = day + fourth_wave_days_length
            
            
        # Record
        if date == '2020-12-07':
            dec7_covid = covid
            if verbose:
                print('...COVID by 7 Dec: {}'.format(dec7_covid))
                    
        if date == '2020-12-31':
            y2020_covid = covid
            if verbose:
                print('...Total 2020 COVID: {}'.format(covid))
                
        if date.split('-')[-1] == '01':
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
                
        if date == '2021-03-31':
            q1_2021_covid = covid
            q1_2021_vaccinated = total_vaccinated
            q1_2021_target = np.round((covid + total_vaccinated) / US_POP * 100, 1)
            if verbose:
                print('...Q1 2021 COVID: {}'.format(q1_2021_covid))
                print('...Q1 2021 Vaccinated: {}'.format(q1_2021_vaccinated))
                print('...Q1 2021 Target: {}'.format(q1_2021_target))

        if date == '2021-06-30':
            q2_2021_covid = covid
            q2_2021_vaccinated = total_vaccinated
            q2_2021_target = np.round((covid + total_vaccinated) / US_POP * 100, 1)
            if verbose:
                print('...Q2 2021 COVID: {}'.format(q2_2021_covid))
                print('...Q2 2021 Vaccinated: {}'.format(q2_2021_vaccinated))
                print('...Q2 2021 Target: {}'.format(q2_2021_target))
            
        if date == '2021-09-30':
            q3_2021_covid = covid
            q3_2021_vaccinated = total_vaccinated
            q3_2021_target = np.round((covid + total_vaccinated) / US_POP * 100, 1)
            if verbose:
                print('...Q3 2021 COVID: {}'.format(q3_2021_covid))
                print('...Q3 2021 Vaccinated: {}'.format(q3_2021_vaccinated))
                print('...Q3 2021 Target: {}'.format(q3_2021_target))
                
        if date == '2021-12-31':
            q4_2021_covid = covid
            q4_2021_vaccinated = total_vaccinated
            q4_2021_target = np.round((covid + total_vaccinated) / US_POP * 100, 1)
            if verbose:
                print('...Q4 2021 COVID: {}'.format(q4_2021_covid))
                print('...Q4 2021 Vaccinated: {}'.format(q4_2021_vaccinated))
                print('...Q4 2021 Target: {}'.format(q4_2021_target))
   
    if not done:
        target_date_collector.append(1000)

    dec_7_2020_covid_collector.append(dec7_covid)
    y2020_covid_collector.append(y2020_covid)
    q1_2021_covid_collector.append(q1_2021_covid)
    q2_2021_covid_collector.append(q2_2021_covid)
    q3_2021_covid_collector.append(q3_2021_covid)
    q4_2021_covid_collector.append(q4_2021_covid)
    q1_2021_vaccinated_collector.append(q1_2021_vaccinated)
    q2_2021_vaccinated_collector.append(q2_2021_vaccinated)
    q3_2021_vaccinated_collector.append(q3_2021_vaccinated)
    q4_2021_vaccinated_collector.append(q4_2021_vaccinated)
    q1_2021_target_collector.append(q1_2021_target)
    q2_2021_target_collector.append(q2_2021_target)
    q3_2021_target_collector.append(q3_2021_target)
    q4_2021_target_collector.append(q4_2021_target)

-
-
## SCENARIO 1/1000 ##
Growth derivative factor: 3.6
Peak growth on: 2020-12-01
Day 0: 2020-12-01
...Total vaccinated now 0 (+0)
...Total vaccinated now 0 (+0)
...Total COVID cases now 13763585.0 (+159667.0 +3918 -0.0 *3.0)
...% of target: 4.2%/70.1%
Day 1: 2020-12-02
...Total vaccinated now 0 (+0)
...Total vaccinated now 0 (+0)
...Total COVID cases now 13924194.0 (+159521.0 +1088 -0.0 *3.8)
...% of target: 4.2%/70.1%
Day 2: 2020-12-03
...Total vaccinated now 0 (+0)
...Total vaccinated now 0 (+0)
...Total COVID cases now 14085699.0 (+159375.0 +2130 -0.0 *3.8)
...% of target: 4.3%/70.1%
Day 3: 2020-12-04
...Total vaccinated now 0 (+0)
...Total vaccinated now 0 (+0)
...Total COVID cases now 14242657.0 (+159229.0 -2271 -0.0 *3.8)
...% of target: 4.3%/70.1%
Day 4: 2020-12-05
...Total vaccinated now 0 (+0)
...Total vaccinated now 0 (+0)
...Total COVID cases now 14405304.0 (+159083.0 +3564 -0.0 *3.8)
...% of target: 4.4%/70.1%
Day 5: 2020-12-06
...Total vaccinated now 0 (+0)
...Total vacc

...Total COVID cases now 45198888.0 (+117831.0 +1824 -0.0 *1.9)
...% of target: 13.8%/70.1%
Day 228: 2021-07-17
...Total vaccinated now 0 (+0)
...Total vaccinated now 0 (+0)
...Total COVID cases now 45318519.0 (+117539.0 +2092 -0.0 *1.9)
...% of target: 13.8%/70.1%
Day 229: 2021-07-18
...Total vaccinated now 0 (+0)
...Total vaccinated now 0 (+0)
...Total COVID cases now 45435946.0 (+117247.0 +180 -0.0 *1.9)
...% of target: 13.8%/70.1%
Day 230: 2021-07-19
...Total vaccinated now 7410.0 (+7410.0)
...Total vaccinated now 7410.0 (+7410.0)
...Total COVID cases now 45548765.0 (+116955.0 -3988 -148.0 *1.9)
...% of target: 13.9%/70.1%
Day 231: 2021-07-20
...Total vaccinated now 15437.0 (+8027.0)
...Total vaccinated now 15437.0 (+8027.0)
...Total COVID cases now 45666915.0 (+116515.0 +1796 -161.0 *1.9)
...% of target: 13.9%/70.1%
Day 232: 2021-07-21
...Total vaccinated now 23464.0 (+8027.0)
...Total vaccinated now 23464.0 (+8027.0)
...Total COVID cases now 45782586.0 (+116062.0 -230 -161.0 *1.9

...Total vaccinated now 19282995.0 (+269176.0)
...Total vaccinated now 19282995.0 (+269176.0)
...Total COVID cases now 51598473.0 (+0 +21 -5384.0 *2.1)
...% of target: 21.6%/70.1%
Day 409: 2022-01-14
...Total vaccinated now 19552171.0 (+269176.0)
...Total vaccinated now 19552171.0 (+269176.0)
...Total COVID cases now 51604745.0 (+0 +6272 -5384.0 *2.1)
...% of target: 21.7%/70.1%
Day 410: 2022-01-15
...Total vaccinated now 19821347.0 (+269176.0)
...Total vaccinated now 19821347.0 (+269176.0)
...Total COVID cases now 51605242.0 (+0 +497 -5384.0 *2.1)
...% of target: 21.8%/70.1%
Day 411: 2022-01-16
...Total vaccinated now 20090523.0 (+269176.0)
...Total vaccinated now 20090523.0 (+269176.0)
...Total COVID cases now 51604930.0 (+0 -312 -5384.0 *2.1)
...% of target: 21.8%/70.1%
Day 412: 2022-01-17
...Total vaccinated now 20359699.0 (+269176.0)
...Total vaccinated now 20359699.0 (+269176.0)
...Total COVID cases now 51599414.0 (+0 -5516 -5384.0 *2.1)
...% of target: 21.9%/70.1%
Day 413: 2022-

...Total vaccinated now 138331815.0 (+1405940.0)
...Total vaccinated now 138331815.0 (+1405940.0)
...Total COVID cases now 51744368.0 (+0 -1134 -28119.0 *2.6)
...% of target: 57.9%/70.1%
Day 580: 2022-07-04
...Total vaccinated now 139737755.0 (+1405940.0)
...Total vaccinated now 139737755.0 (+1405940.0)
...Total COVID cases now 51743518.0 (+0 -850 -28119.0 *2.6)
...% of target: 58.3%/70.1%
Day 581: 2022-07-05
...Total vaccinated now 141143695.0 (+1405940.0)
...Total vaccinated now 141143695.0 (+1405940.0)
...Total COVID cases now 51744128.0 (+0 +610 -28119.0 *2.6)
...% of target: 58.8%/70.1%
Day 582: 2022-07-06
...Total vaccinated now 142549635.0 (+1405940.0)
...Total vaccinated now 142549635.0 (+1405940.0)
...Total COVID cases now 51740808.0 (+0 -3320 -28119.0 *2.6)
...% of target: 59.2%/70.1%
Day 583: 2022-07-07
...Total vaccinated now 143955575.0 (+1405940.0)
...Total vaccinated now 143955575.0 (+1405940.0)
...Total COVID cases now 51744773.0 (+0 +3965 -28119.0 *2.6)
...% of target:

...% of target: 119.5%/70.1%
Day 708: 2022-11-09
...Total vaccinated now 342850959.0 (+2486341.0)
...Total vaccinated now 342850959.0 (+2486341.0)
...Total COVID cases now 51874685.0 (+0 -2238 -49727.0 *3.9)
...% of target: 120.3%/70.1%
Day 709: 2022-11-10
...Total vaccinated now 345337300.0 (+2486341.0)
...Total vaccinated now 345337300.0 (+2486341.0)
...Total COVID cases now 51871810.0 (+0 -2875 -49727.0 *3.9)
...% of target: 121.0%/70.1%
Day 710: 2022-11-11
...Total vaccinated now 347823641.0 (+2486341.0)
...Total vaccinated now 347823641.0 (+2486341.0)
...Total COVID cases now 51872428.0 (+0 +618 -49727.0 *3.9)
...% of target: 121.8%/70.1%
Day 711: 2022-11-12
...Total vaccinated now 350309982.0 (+2486341.0)
...Total vaccinated now 350309982.0 (+2486341.0)
...Total COVID cases now 51876093.0 (+0 +3665 -49727.0 *3.9)
...% of target: 122.5%/70.1%
Day 712: 2022-11-13
...Total vaccinated now 352796323.0 (+2486341.0)
...Total vaccinated now 352796323.0 (+2486341.0)
...Total COVID cases n

...

-
-
## SCENARIO 191/1000 ##
Growth derivative factor: 0.3
Peak growth on: 2021-01-19
...COVID by 7 Dec: 15492382.0
...Total 2020 COVID: 35128587.0
...Q1 2021 COVID: 145187352.0
...Q1 2021 Vaccinated: 0
...Q1 2021 Target: 44.2
...Q2 2021 COVID: 233993749.0
...Q2 2021 Vaccinated: 0
...Q2 2021 Target: 71.3
...Q3 2021 COVID: 288556850.0
...Q3 2021 Vaccinated: 7125091.0
...Q3 2021 Target: 90.1
...Q4 2021 COVID: 312899014.0
...Q4 2021 Vaccinated: 27190867.0
...Q4 2021 Target: 103.6
-
-
## SCENARIO 201/1000 ##
Growth derivative factor: 12.2
Peak growth on: 2020-12-13
...COVID by 7 Dec: 14737345.0
...Total 2020 COVID: 18679676.0
...Q1 2021 COVID: 33285469.0
...Q1 2021 Vaccinated: 0
...Q1 2021 Target: 10.1
...Q2 2021 COVID: 47394671.0
...Q2 2021 Vaccinated: 0
...Q2 2021 Target: 14.4
...Q3 2021 COVID: 55647863.0
...Q3 2021 Vaccinated: 12713190.0
...Q3 2021 Target: 20.8
...Q4 2021 COVID: 55727626.0
...Q4 2021 Vaccinated: 60639405.0
...Q4 2021 Target: 35.5
-
-
## SCENARIO 211/1000 ##
Growth deriva

-
-
## SCENARIO 371/1000 ##
Growth derivative factor: 7.0
Peak growth on: 2020-12-20
...COVID by 7 Dec: 14736894.0
...Total 2020 COVID: 18931232.0
...Q1 2021 COVID: 34493774.0
...Q1 2021 Vaccinated: 0
...Q1 2021 Target: 10.5
...Q2 2021 COVID: 49538286.0
...Q2 2021 Vaccinated: 0
...Q2 2021 Target: 15.1
...Q3 2021 COVID: 60994205.0
...Q3 2021 Vaccinated: 6136805.0
...Q3 2021 Target: 20.5
...Q4 2021 COVID: 61229751.0
...Q4 2021 Vaccinated: 25339258.0
...Q4 2021 Target: 26.4
-
-
## SCENARIO 381/1000 ##
Growth derivative factor: 28.6
Peak growth on: 2020-12-01
...COVID by 7 Dec: 14703603.0
...Total 2020 COVID: 18508367.0
...Q1 2021 COVID: 32618770.0
...Q1 2021 Vaccinated: 0
...Q1 2021 Target: 9.9
...Q2 2021 COVID: 46614645.0
...Q2 2021 Vaccinated: 0
...Q2 2021 Target: 14.2
...Q3 2021 COVID: 55559020.0
...Q3 2021 Vaccinated: 11364393.0
...Q3 2021 Target: 20.4
...Q4 2021 COVID: 55662812.0
...Q4 2021 Vaccinated: 52805143.0
...Q4 2021 Target: 33.0
-
-
## SCENARIO 391/1000 ##
Growth derivative f

-
-
## SCENARIO 551/1000 ##
Growth derivative factor: 5.0
Peak growth on: 2020-12-01
...COVID by 7 Dec: 14710370.0
...Total 2020 COVID: 18474495.0
...Q1 2021 COVID: 31845815.0
...Q1 2021 Vaccinated: 0
...Q1 2021 Target: 9.7
...Q2 2021 COVID: 44169699.0
...Q2 2021 Vaccinated: 0
...Q2 2021 Target: 13.5
...Q3 2021 COVID: 50409645.0
...Q3 2021 Vaccinated: 9135521.0
...Q3 2021 Target: 18.1
...Q4 2021 COVID: 50488482.0
...Q4 2021 Vaccinated: 41447672.0
...Q4 2021 Target: 28.0
-
-
## SCENARIO 561/1000 ##
Growth derivative factor: 34.1
Peak growth on: 2020-12-01
...COVID by 7 Dec: 14722687.0
...Total 2020 COVID: 18507506.0
...Q1 2021 COVID: 32444501.0
...Q1 2021 Vaccinated: 0
...Q1 2021 Target: 9.9
...Q2 2021 COVID: 46322594.0
...Q2 2021 Vaccinated: 0
...Q2 2021 Target: 14.1
...Q3 2021 COVID: 59075060.0
...Q3 2021 Vaccinated: 2646687.0
...Q3 2021 Target: 18.8
...Q4 2021 COVID: 61865893.0
...Q4 2021 Vaccinated: 14601751.0
...Q4 2021 Target: 23.3
-
-
## SCENARIO 571/1000 ##
Growth derivative fac

-
-
## SCENARIO 731/1000 ##
Growth derivative factor: 29.4
Peak growth on: 2020-12-01
...COVID by 7 Dec: 14719227.0
...Total 2020 COVID: 18536061.0
...Q1 2021 COVID: 33032935.0
...Q1 2021 Vaccinated: 0
...Q1 2021 Target: 10.1
...Q2 2021 COVID: 47751145.0
...Q2 2021 Vaccinated: 0
...Q2 2021 Target: 14.5
...Q3 2021 COVID: 59162873.0
...Q3 2021 Vaccinated: 6351287.0
...Q3 2021 Target: 20.0
...Q4 2021 COVID: 59335248.0
...Q4 2021 Vaccinated: 33914933.0
...Q4 2021 Target: 28.4
-
-
## SCENARIO 741/1000 ##
Growth derivative factor: 3.5
Peak growth on: 2020-12-29
...COVID by 7 Dec: 14750820.0
...Total 2020 COVID: 19180240.0
...Q1 2021 COVID: 35822615.0
...Q1 2021 Vaccinated: 0
...Q1 2021 Target: 10.9
...Q2 2021 COVID: 50932444.0
...Q2 2021 Vaccinated: 0
...Q2 2021 Target: 15.5
...Q3 2021 COVID: 62597018.0
...Q3 2021 Vaccinated: 4995395.0
...Q3 2021 Target: 20.6
...Q4 2021 COVID: 62934867.0
...Q4 2021 Vaccinated: 24831754.0
...Q4 2021 Target: 26.7
-
-
## SCENARIO 751/1000 ##
Growth derivative f

-
-
## SCENARIO 911/1000 ##
Growth derivative factor: 12.8
Peak growth on: 2021-01-05
...COVID by 7 Dec: 14725666.0
...Total 2020 COVID: 18781149.0
...Q1 2021 COVID: 34397487.0
...Q1 2021 Vaccinated: 0
...Q1 2021 Target: 10.5
...Q2 2021 COVID: 49658889.0
...Q2 2021 Vaccinated: 0
...Q2 2021 Target: 15.1
...Q3 2021 COVID: 61733830.0
...Q3 2021 Vaccinated: 5944304.0
...Q3 2021 Target: 20.6
...Q4 2021 COVID: 62059960.0
...Q4 2021 Vaccinated: 29029680.0
...Q4 2021 Target: 27.8
-
-
## SCENARIO 921/1000 ##
Growth derivative factor: 17.7
Peak growth on: 2021-01-19
...COVID by 7 Dec: 14724718.0
...Total 2020 COVID: 18864512.0
...Q1 2021 COVID: 35398251.0
...Q1 2021 Vaccinated: 0
...Q1 2021 Target: 10.8
...Q2 2021 COVID: 52557839.0
...Q2 2021 Vaccinated: 0
...Q2 2021 Target: 16.0
...Q3 2021 COVID: 65982890.0
...Q3 2021 Vaccinated: 8093149.0
...Q3 2021 Target: 22.6
...Q4 2021 COVID: 66148632.0
...Q4 2021 Vaccinated: 49204890.0
...Q4 2021 Target: 35.1
-
-
## SCENARIO 931/1000 ##
Growth derivative 

## Total Worldwide COVID by 7 Dec

For calibration

In [127]:
get_metaculus_range(dec_7_2020_covid_collector)

{'lower25': 14714429.0, 'mean': 14721121.0, 'upper75': 14728095.75}

## By EOY 2020

In [128]:
get_metaculus_range(y2020_covid_collector)

{'lower25': 18529822.5, 'mean': 18618002.5, 'upper75': 18719872.5}

## By EOQ Q1 2021

In [129]:
get_metaculus_range(q1_2021_covid_collector)

{'lower25': 32667808.0, 'mean': 33365566.0, 'upper75': 34358801.5}

In [130]:
get_metaculus_range(q1_2021_vaccinated_collector)

{'lower25': 0.0, 'mean': 0.0, 'upper75': 0.0}

In [131]:
get_metaculus_range(q1_2021_target_collector)

{'lower25': 10.0, 'mean': 10.2, 'upper75': 10.5}

## By EOQ Q2 2021

In [132]:
get_metaculus_range(q2_2021_covid_collector)

{'lower25': 46732722.25, 'mean': 48114986.5, 'upper75': 50233253.0}

In [133]:
get_metaculus_range(q2_2021_vaccinated_collector)

{'lower25': 0.0, 'mean': 0.0, 'upper75': 0.0}

In [134]:
get_metaculus_range(q2_2021_target_collector)

{'lower25': 14.2, 'mean': 14.7, 'upper75': 15.3}

## By EOQ Q3 2021

In [135]:
get_metaculus_range(q3_2021_covid_collector)

{'lower25': 56445342.25, 'mean': 59294932.5, 'upper75': 62630036.0}

In [136]:
get_metaculus_range(q3_2021_vaccinated_collector)

{'lower25': 5729562.75, 'mean': 8103282.0, 'upper75': 10090934.0}

In [137]:
get_metaculus_range(q3_2021_target_collector)

{'lower25': 19.7, 'mean': 20.6, 'upper75': 21.6}

## By EOY 2021

In [138]:
get_metaculus_range(q4_2021_covid_collector)

{'lower25': 56592247.75, 'mean': 59609154.0, 'upper75': 63680154.0}

In [139]:
get_metaculus_range(q4_2021_vaccinated_collector)

{'lower25': 26178107.5, 'mean': 35908841.5, 'upper75': 45635564.25}

In [140]:
get_metaculus_range(q4_2021_target_collector)

{'lower25': 26.5, 'mean': 29.4, 'upper75': 32.6}

## Target Date

In [141]:
'target not exceeded by 2022-12-21 in {} of scenarios'.format(print_pct(sum([x == 1000 for x in target_date_collector]) / len(target_date_collector)))

'target not exceeded by 2022-12-21 in <1.0% of scenarios'

In [142]:
results = get_metaculus_range([x for x in target_date_collector if x < 1000])
results['lower25'] = str((datetime.strptime(SIM_START_DATE, '%Y-%m-%d') + timedelta(days=results['lower25'])).date())
results['mean'] = str((datetime.strptime(SIM_START_DATE, '%Y-%m-%d') + timedelta(days=results['mean'])).date())
results['upper75'] = str((datetime.strptime(SIM_START_DATE, '%Y-%m-%d') + timedelta(days=results['upper75'])).date())
results

{'lower25': '2022-04-21', 'mean': '2022-05-14', 'upper75': '2022-06-12'}